# Chat Completion API

In [ ]:
!pip install openai

## 환경설정
1. openai사이트에서 API KEY 발급
2. google colab - secret탭에 `OPENAI_API_KEY` 등록

In [2]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [3]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "너는 아주 친절하고, 많은 도움을 주는 챗봇이야~"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "안녕~ 내 이름은 차은우야~"
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [ ]:
# ChatCompletion(choices=[Choice(message(content))])
response.choices[0].message.content

'안녕하세요, 차은우님! 반갑습니다. 어떻게 도와드릴까요?'

## 프롬프팅의 기본구성

https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/

1. Instruction 지시사항
2. Context 문맥
3. Input Data/Example 입력/예시
4. Output Indicator 출력지시

## 기사 제목 교정

- 기자들이 송고한 기사에서 제목을 추출하고, 표현 조정
- 프랑스AFP 속보시스템에서 도입되어 사용


In [ ]:
title_before = '테이의 FM 개꿀 라디오 방송에 주목해주세요.'
title_before = '졸라 빡센 작업으로 끼니 거르기 일쑤인 노동자들의 애환'

# 역할/페르소나/지시사항
system_message = """
기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조등에 있어서 교정작업을 수행해 주세요.

- 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
- 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
- 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
- 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

### Steps ###
1. 기사제목을 읽고 주요내용을 이해하세요.
2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
3. 맞춤법, 문법, 의미 전달의 정확성등을 점검하고 적절히 수정하세요.
4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈수 있는지 점검하고 매우 간결하게 정리하세요.


### Output Format ###
기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

- 원래 제목: [기사 원래 제목]
- 교정 제목: [교정 기사 제목]

### Examples ###
- 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
- 교정 제목: "서울 대형 화재, 수백명 대피"

- 원래 제목: "전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다."
- 교정 제목: "전기차 판매량 급감에 웃는 내연차회사들"

### Extra Instructions ###
- 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
- 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
- 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

"""

user_message = f"""
다음 기사제목을 교정해주세요.

제목: {title_before}
"""

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": system_message
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": user_message
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
print(response.choices[0].message.content)

- 원래 제목: 졸라 빡센 작업으로 끼니 거르기 일쑤인 노동자들의 애환
- 교정 제목: 힘든 작업으로 끼니를 거르는 노동자들의 고충


In [ ]:
# 함수 리팩토링

def correct_news_title(title_before, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조등에 있어서 교정작업을 수행해 주세요.

    - 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
    - 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
    - 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
    - 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

    ### Steps ###
    1. 기사제목을 읽고 주요내용을 이해하세요.
    2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
    3. 맞춤법, 문법, 의미 전달의 정확성등을 점검하고 적절히 수정하세요.
    4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈수 있는지 점검하고 매우 간결하게 정리하세요.


    ### Output Format ###
    기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

    - 원래 제목: [기사 원래 제목]
    - 교정 제목: [교정 기사 제목]

    ### Examples ###
    - 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
    - 교정 제목: "서울 대형 화재, 수백명 대피"

    - 원래 제목: "전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다."
    - 교정 제목: "전기차 판매량 급감에 웃는 내연차회사들"

    ### Extra Instructions ###
    - 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
    - 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
    - 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

    """

    user_message = f"""
    다음 기사제목을 교정해주세요.

    제목: {title_before}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

title_before = '주말 미친 폭우 예상, 모두들 무사하시길~'
output = correct_news_title(title_before)
print(output)

- 원래 제목: 주말 미친 폭우 예상, 모두들 무사하시길~
- 교정 제목: "주말 폭우 예보, 안전에 유의하세요"


## 연애코치 ReAct

In [4]:
def dating_coach(prompt, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    << System Instruction >>
    어떤 상황에서든 최고의 논리적/감성적 관점을 적용하는 연애코치로써 사용자의 고민을 해결해 주세요.

    << Output Format >>
    1. 상황분석:

    2. 행동계획:

    3. 실행:
    """

    user_message = f"""
    사용자 현재상황:
    {prompt}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [7]:
print(dating_coach('저 이번주말에 여자친구와 100일인데, 기억에 남을 선물을 하고 싶어요. 뭐가 좋을까요?'))

1. 상황분석:
   - 당신은 여자친구와의 100일을 기념하기 위해 특별하고 기억에 남을 선물을 준비하고자 합니다.
   - 중요한 기념일이므로, 그녀의 취향과 두 사람의 관계에서의 특별한 순간을 고려해야 합니다.
   - 여친이 좋아할만한 선물은 감정적으로 의미가 있어야 하며, 소중한 추억을 남길 수 있는 요소가 포함되어야 합니다.

2. 행동계획:
   - 여자친구의 취향과 관심사 분석하기:
     - 그녀가 좋아하는 브랜드, 색깔, 취미 등을 떠올려 보세요.
   - 직접적인 선물 외에도 특별한 경험을 선물하는 것이 좋습니다.
     - 예를 들어, 둘만의 소중한 데이트 코스를 계획하거나 함께 하는 액티비티를 고려해보세요.
   - 기념일의 의미를 담은 편지나 카드 작성하기:
     - 기념일에 대한 감정을 표현하고, 서로에 대한 사랑과 감사의 마음을 담아보세요.

3. 실행:
   - 그녀가 좋아하는 취미에 맞춰서 맞춤형 선물: 예를 들어, 그녀가 책을 좋아한다면 첫 번째 기념일에 주 마음에 드는 책에 당신의 메시지를 적어 선물할 수 있습니다.
   - 특별한 날짜에 대한 회상: 두 분의 특별한 장소에서 다시 한번 데이트를 하며 추억을 되새기는 시간을 가져보세요.
   - 마지막으로, 정성스럽게 만든 편지나 메시지를 함께 전달하여 감정의 깊이를 더해 주세요. 

이런 방식으로 준비한다면, 선물은 물론이고 기념일 자체가 더욱 특별하고 잊지 못할 경험이 될 것입니다.


In [8]:
print(dating_coach('제가 어제 친구들이 놀다가 여자친구랑 연락을 잘 못해서, 단단히 삐져있는 상황입니다. 이 위기를 어떻게 극복해야 할까요?'))

1. 상황분석:
   - 여자친구는 당신이 친구들과 보낸 시간에 소외감을 느끼고, 연락이 끊어진 것에 대해 불쾌감을 가지고 있습니다.
   - 이로 인해 감정이 상했고, 현재의 상황에 대해 불안이나 서운함을 느끼고 있을 가능성이 높습니다. 즉, 소통 부족이 문제의 핵심입니다.
   - 당신도 그녀의 기분을 상하게 하고 싶지 않았을 것이며, 이 상황을 해결하고 싶다는 마음이 드는 것도 이해할 수 있습니다.

2. 행동계획:
   - 먼저, 그녀와 충분한 대화를 통해 그녀의 기분을 이해할 필요가 있습니다.
   - 진심 어린 사과와 함께 왜 연락을 하지 못했는지를 설명하고, 그녀의 감정을 존중하는 모습을 보여야 합니다.
   - 사과 후에는 그녀의 감정에 대해 진솔하게 대화하며 서로의 기대와 불만을 나누는 시간을 가집니다.
   - 좀 더 긍정적인 감정을 회복하기 위한 작은 데이트 아이디어를 구상해보고, 그에 대한 제안을 즐겁게 할 수 있도록 합니다.

3. 실행:
   - 먼저 메시지를 보내 사과의 뜻을 전하세요. 예를 들어, "지난 번에 연락을 못해서 미안해. 너의 기분이 어땠는지 정말 궁금해."와 같이 시작할 수 있습니다.
   - 그녀의 반응에 따라 대화를 이어가세요. 그녀가 이야기하고 싶어하는 주제를 중시하며, 그녀의 기분을 받아들이고 이해하는 모습을 보여줍니다.
   - 가능하다면 직접 만나서 대화하는 시간을 가지는 것이 좋습니다. 서로 얼굴을 마주하고 이야기하면 감정이 더 쉽게 전달될 수 있습니다.
   - 대화 중에는 다음에 함께 하고 싶은 활동이나 데이트를 제안하여 그녀에게 당신이 그녀를 소중히 여긴다는 것을 알려주세요.


## 냉털마스터 ReAct
- 사용자는 냉장고에 남아있는 음식재료를 알려주면, LLM은 이를 바탕으로 어떤 음식을 만들지를 조언해준다.
- Reasoning/Action을 끌어낼수 있는 적절한 프롬프팅을 작성하세요.


In [13]:
def fridge_raid_master(user_foods, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
당신은 사용자 냉장고의 재료를 가지고 최고의 음식을 만들 수 있는 레시피를 추천하는 챗봇입니다.
현재 상황을 분석하고 실행계획을 세우며 추가 내용이 있을지 확인하는 꼼꼼함을 보여주세요.

## 지시사항

### 상황분석
1. **재료 확인**: 냉장고에 있는 재료 목록을 작성하고, 각 재료의 신선함과 사용 가능 시간을 파악합니다.
2. **요리 컨셉 설정**: 재료를 고려하여 요리의 주제를 정하고 잠재적인 요리 아이디어를 계획합니다.

### 실행계획
3. **목표 변경**: 사용 가능한 재료로 만들고자 하는 요리의 목표를 설정하고, 필요한 정보나 추가 자료를 조사합니다.
4. **전략 개발**: 단계별로 목표를 달성하기 위한 전반적인 요리 계획을 세웁니다.
    단, 사용자가 따라하기 쉽게 단계별 가이드를 작성해야 합니다.

### 검증 및 추가내용 확인
5. **레시피 검토**: 작성된 레시피를 검토하여 정보가 명확하고 완전한지 확인합니다.
6. **수정**: 필요 시 수정하여 최종 레시피를 완성합니다.

## 출력형식

1. 상황분석:
   - 재료 목록 및 상태
   - 요리 컨셉 및 아이디어

2. 실행계획:
   - 목표 및 세부 계획
   - 필요한 추가 자료 및 전략

3. 검증 및 추가내용 확인
   - 레시피의 정확성 검토
   - 수정 및 편집

4. 최종레시피
   - 사용자가 따라하기 쉽게 목록으로 작성
   - 필요한 준비물로 별도록 작성할것!

## Examples

- **Input**:
    사용자의 냉장고에는 현재 [무, 파, 두부]이/가 있습니다.
-   **Output**:
  1. 상황분석: 오늘은 냉장고에 있는 무, 파, 두부를 이용해 한식 스프를 만들어보겠습니다...
  2. 실행계획: 먼저 무와 파를 얇게 썰어 냄비에 넣고 물을 부어...
  3. 검증 및 추가내용 확인: 레시피를 확인한 결과, 무와 두부의 사용 방법에 대한 설명이 추가로 필요합니다...

## Notes

- 요리의 문화적 관련성과 감수성을 고려합니다.
- 레시피는 예상 독자의 요리 지식 수준에 맞게 조정됩니다.

    """

    user_message = f"""
    사용자의 냉장고에는 현재 {user_foods}이/가 있습니다.
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [15]:
from IPython.display import Markdown

user_foods = ['소고기', '양파', '표고버섯', '명란젓', '된장']
display(Markdown(fridge_raid_master(user_foods)))

1. 상황분석:
   - 재료 목록 및 상태:
     - 소고기: 신선함이 좋음, 사용 가능 시간: 2-3일 남음
     - 양파: 신선함 양호, 사용 가능 시간: 1주일 남음
     - 표고버섯: 신선함 양호, 사용 가능 시간: 3-4일 남음
     - 명란젓: 유통기한 내에 있음, 사용 가능 시간: 2개월 이상
     - 된장: 유통기한 내에 있음, 사용 가능 시간: 1년 이상
  
   - 요리 컨셉 및 아이디어:
     오늘의 요리는 '국물 요리'를 주제로 하여, 소고기와 표고버섯으로 만든 된장찌개에 명란젓을 포함하여 풍미를 더해보겠습니다. 또한, 양파를 사용하여 맛을 깊게 할 계획입니다.

2. 실행계획:
   - 목표 및 세부 계획: 
     - 소고기와 표고버섯, 양파를 주재료로 한 된장찌개 만들기.
     - 명란젓을 곁들여 국물의 맛을 한층 업그레이드.
  
   - 필요한 추가 자료 및 전략:
     - 된장찌개의 기본 레시피 및 조리법을 확인하고, 명란젓을 추가하는 방법을 구체적으로 정리합니다.

3. 검증 및 추가내용 확인:
   - 레시피의 정확성 검토: 기본된장찌개 레시피에 명란젓 추가하는 부분을 명확하게 적어야 합니다. 또한, 조리 시간을 확인해 쉽게 따라할 수 있도록 할 수 있습니다.

4. 최종레시피:
   - 준비물:
     - 소고기 200g
     - 양파 1개
     - 표고버섯 100g
     - 된장 2큰술
     - 명란젓 1개 (50g)
     - 물 4컵 (약 1리터)

   - 요리 단계:
     1. 양파는 채썰고, 표고버섯은 슬라이스로 자릅니다.
     2. 소고기는 적당한 크기로 잘라 준비합니다.
     3. 냄비에 물을 붓고, 소고기를 넣어 끓입니다. (중불)
     4. 소고기가 익으면 양파와 표고버섯을 넣고 다시 끓입니다.
     5. 된장을 풀어 넣고, 원하는 간에 맞게 조절합니다. (중불에서 10분 정도 끓입니다)
     6. 마지막으로 명란젓을 넣고, 잠시 더 끓여서 완성합니다.
     7. 그릇에 담아 따뜻하게 즐기세요!

이 레시피는 쉽게 따라할 수 있으며, 필요한 재료들이 잘 활용됩니다. 만약 추가적으로 알고 싶은 정보가 있다면 말씀해 주세요!

## 면접질문 생성 JSON 출력


In [32]:
def job_interview(job_posting, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
당신은 머신러닝/딥러닝/LLM/AI서비스개발의 전문가로써, 해당분야의 으뜸가는 면접관입니다.
매번 그룹사의 인재를 발굴하기 위해 면접질문/모범답안을 작성하고 있습니다.

<<지시사항>>
- 사용자가 제출한 채용공고의 내용을 바탕으로 예상면접질문과 답변을 작성해주세요.
- 하드스킬과 소프트스킬 두개의 섹션으로 나누어 작성해주세요.
- 각 스킬별로 질문/답변을 3개씩 만들어주세요.

<<출력형식>>
출력은 json형식으로만 반환되어야 합니다.

{{
    "hard_skill": [
        {{
            "question": "",
            "answer": "",
        }}
    ],
    "soft_skill": [
        {{
            "question": "",
            "answer": "",
        }}
    ]
}}
"""

    user_message = f"""
채용공고:
{job_posting}
"""

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "json_object"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [36]:
def job_interview(job_posting, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
당신은 머신러닝/딥러닝/LLM/AI서비스개발의 전문가로써, 해당분야의 으뜸가는 면접관입니다.
매번 그룹사의 인재를 발굴하기 위해 면접질문/모범답안을 작성하고 있습니다.
"""

    user_message = f"""

<<지시사항>>
- 사용자가 제출한 채용공고의 내용을 바탕으로 예상면접질문과 답변을 작성해주세요.
- 하드스킬과 소프트스킬 두개의 섹션으로 나누어 작성해주세요.
- 각 스킬별로 질문/답변을 3개씩 만들어주세요.

<<출력형식>>
출력은 json형식으로만 반환되어야 합니다.

{{
    "hard_skill": [
        {{
            "question": "",
            "answer": "",
        }}
    ],
    "soft_skill": [
        {{
            "question": "",
            "answer": "",
        }}
    ]
}}

<< 채용공고 >>
{job_posting}
"""

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "json_object"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [37]:
job_posting = """
| 모집부문                      | 담당업무                                                                                                                                                                                                                                                                                               | 자격요건 및 필수사항                                                                                                                                                                                                                                                                                                                                                                 |
| ------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **AI 인프라 구축 (ML/DEVOps)** | 1. AI 모델의 배포를 위한 인프라 구축 및 운영<br>2. 알고리즘 개발자에 대한 기술 지원<br>3. 머신러닝 인프라의 설계 및 개발, 운영<br>4. 모니터링, 로깅 시스템의 개발 및 구현<br>5. 머신러닝 시스템의 학습, 배포, 테스트 환경 구축, 운영 및 자동화<br>6. 오픈소스 기반의 플랫폼 통합 및 내재화 구축<br>(MLflow, LakeFS, CVAT, BentoML 등)<br>7. 보안, 인증, 권한 관리 등 운영 정책 및 관리자 기능 구축<br>8. 시스템과 프로세스에 대한 문서 작업 | **학력**<br>- 대졸이상(4년제)<br><br>**경력**<br>- 신입<br><br>**필수사항**<br>- Python 프로그래밍 능력<br>- 컨테이너 오케스트레이션 툴 활용 능력(Docker, Kubernetes 등)<br>- Jenkins, ArgoCD, Gitlab 등 구축 및 운영 경험<br><br>**우대사항**<br>- DevOps 환경 구축 혹은 DevOps 환경에서 개발 및 배포 경험 보유자<br>- MLflow, Kubeflow, ML 모델 서빙 플랫폼 등과 관련된 실무 경험 보유자<br>- Kubernetes, Docker, GitOps, CI/CD, Vault, MinIO 등 클라우드 네이티브 인프라 경험 보유자 |
"""
output = job_interview(job_posting)
print(type(output))
print(output)

<class 'str'>
{
    "hard_skill": [
        {
            "question": "Python과 관련된 프로젝트 경험에 대해 설명해주실 수 있나요?",
            "answer": "물론입니다. 제가 고객 데이터를 분석하는 프로젝트에서 Python을 사용했습니다. 데이터 전처리와 분석을 위해 Pandas와 NumPy를 이용했고, 데이터 시각화를 위해 Matplotlib과 Seaborn을 사용했습니다. 이 경험을 통해 데이터 처리 흐름과 시각화의 중요성을 이해하게 되었습니다."
        },
        {
            "question": "컨테이너 오케스트레이션 툴인 Kubernetes를 사용해 본 경험에 대해 말씀해 주세요.",
            "answer": "네, 이전 프로젝트에서 Kubernetes를 사용해 애플리케이션을 배포하고 관리했습니다. 여러 마이크로서비스를 컨테이너화하여 클러스터에서 자동으로 스케일링하고 롤아웃하는 경험이 있습니다. Helm 차트를 사용하여 배포를 간소화하고 관리했으며, 서비스의 가용성을 높이기 위한 로드 밸런싱 설정도 진행했습니다."
        },
        {
            "question": "CI/CD 도구인 Jenkins를 활용한 경험을 말해주세요.",
            "answer": "Jenkins를 사용하여 자동화된 배포 파이프라인을 설계하고 구현한 경험이 있습니다. 각 커밋 후 자동으로 테스트가 실행되고, 성공적으로 테스트를 통과한 코드만 생산 환경에 배포되도록 설정했습니다. 이렇게 함으로써 개발과 배포의 효율성을 크게 향상시킬 수 있었습니다."
        }
    ],
    "soft_skill": [
        {
            "question": "팀원들과의 협업에서 가장 중요하게 생각하는 요소는 무엇인가요?",
            "answer": "저는 커뮤니케이션이 가장 중요하다

In [34]:
import json

# 응답데이터가 json형식이 아니라면 오류발생!
data = json.loads(output)
print(type(data))

hard_skill_qa = data['hard_skill']
soft_skill_qa = data['soft_skill']
print(hard_skill_qa)
print(soft_skill_qa)

<class 'dict'>
[{'question': 'Python 프로그래밍을 사용한 머신러닝 인프라 구축 경험에 대해 설명해 주세요.', 'answer': '저는 Python을 사용하여 머신러닝 모델 학습 및 배포 환경을 구축한 경험이 있습니다. 예를 들어, 특정 프로젝트에서는 PyTorch를 이용해 모델을 개발한 후, Flask를 사용해 REST API로 배포했습니다. Docker를 활용해 컨테이너화하여 손쉽게 다양한 환경에서 실행할 수 있도록 했습니다.'}, {'question': 'Docker와 Kubernetes를 이용한 컨테이너 오케스트레이션 경험에 대해 말해 주세요.', 'answer': '컨테이너 오케스트레이션에 있어 Docker를 사용해 각각의 서비스를 컨테이너화한 후, Kubernetes를 통해 클러스터 환경에서 관리했습니다. Helm Chart를 이용해 배포 자동화를 구현했으며, 모니터링과 스케일링을 통해 안정적인 서비스를 제공할 수 있었습니다.'}, {'question': 'Jenkins를 사용한 CI/CD 파이프라인 구축 사례를 말씀해 주세요.', 'answer': '저는 Jenkins를 사용해 CI/CD 파이프라인을 구축한 경험이 있습니다. 코드 변경 시 자동으로 테스트와 빌드를 수행하도록 설정하였고, 성공적으로 빌드가 완료될 경우 Kubernetes 클러스터에 자동 배포하는 흐름을 만들었습니다. 이로 인해 배포 시간과 오류가 크게 줄어들었습니다.'}]
[{'question': '기술 지원을 할 때 중요한 소통 능력에 대해 어떻게 생각하나요?', 'answer': '기술 지원을 할 때, 상대방의 질문을 정확하게 이해하고, 그에 맞는 명확한 답변을 줄 수 있는 것이 중요합니다. 저는 항상 상대의 배경 기초 지식을 고려하여 설명할 내용을 조절하며, 비유나 예시를 들어 이해를 돕고 있습니다.'}, {'question': '팀 내에서 발생할 수 있는 갈등 상황을 어떻게 해결할 것인가요?', 'answer': '갈등 상황이 발생하면 먼저 문제의